### Import cell

In [1]:

import os
import sys
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from torch import nn
import matplotlib.pyplot as plt
from utils import (load_nerl_data,
    get_normalized_adj,
    get_Laplace,
    calculate_random_walk_matrix,
    test_error_cap,
    test_error
)
import random
import copy
import scipy
from scipy.io import loadmat
import math
import pandas as pd
from basic_structure import D_GCN, C_GCN, K_GCN,IGNNK
from basic_process import *
from sklearn.manifold import TSNE
import os
import pandas as pd
from sklearn import linear_model
from pandas import read_csv 

2024-03-25 18:38:18.988634: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-25 18:38:19.012294: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-25 18:38:19.393363: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from utils import *

In [3]:
n_o_n_m = 20 #sampled space dimension

h = 24 #sampled time dimension

z = 100 #hidden dimension for graph convolution

K = 1 #If using diffusion convolution, the actual diffusion convolution step is K+1

n_m = 20 #number of mask node during training

N_u = 50 #target locations, N_u locations will be deleted from the training data

Max_episode = 750 #max training episode

learning_rate = 0.0001 #the learning_rate for Adam optimizer

batch_size = 8

E_maxvalue=80

STmodel = IGNNK(h, z, K)
STmodel.load_state_dict(torch.load('model/IGNNK_ushcn_750iter_2023-11-26 12:25:53.pth'))

<All keys matched successfully>

In [4]:
#Read all data
pathroot='data/NDBC/all_stations'
PATH_ROOT = os.getcwd()
ROOT = os.path.join(PATH_ROOT, pathroot)
filenames = os.listdir(ROOT)
# Sort all files
filenames.sort()
data = []
for i in filenames:
    PATH_CSV = os.path.join(ROOT, i)
    with open(PATH_CSV, 'r') as file:
# Use splitlines() to divide contents in the documents into lists.
        content_list = file.read().splitlines()
# Transform lists into Numpy.
    content_matrix = np.array([list(map(float, line.split())) for line in content_list])
    data.append(content_matrix)
data = np.array(data).transpose(1, 2, 0)
X_raw=data

In [5]:
Station_info=pd.read_csv('data/NDBC/Station_info.csv')

### Experiment 1: Raw setting on IGNNK without training

In [6]:
for j in range(5,X_raw.shape[1]-1):
        NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
        NDBC_long=pd.DataFrame(Station_info.iloc[:,3])
        NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
        Adj_dist=adj_dist(NDBC_lat,NDBC_long)
        X_raw_0=X_raw[:,j,:]#GET the first feature
        #print(X_raw_0.shape)
        # Count the num of missing values in each column
        missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
        # print results
       # print("Incomplete data number in each column：", missing_counts_per_column)
        #Get the index if the value is not zero
        # Find the columns where missing values exist.
        columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

        # Get the column numbers when missing value exist.
        missing_columns = np.where(columns_with_missing_data)[0]

        #Get new data after deletion.
        #Delete those columns(Stations) if there is not any features.
        result = np.delete(X_raw_0, missing_columns, axis=1)
        result = (result - np.min(result)) / (np.max(result) - np.min(result))
        #Mean results---for SNE
        mean_result_axis_1=np.mean(result,axis=0)
        mean_result_axis_1=mean_result_axis_1.reshape(-1,1)        #Get new adjacency matrix after deletion.
        NDBC_long=NDBC_long.transpose()
        NDBC_long_aft=NDBC_long.drop(columns=missing_columns)
        # Reconstruct the index in the DataFrame and get the longitude after deletion.
        NDBC_long_aft = NDBC_long_aft.reset_index(drop=True)
        Temp_long=NDBC_long_aft.values

        NDBC_lat=NDBC_lat.transpose()
        NDBC_lat_aft=NDBC_lat.drop(columns=missing_columns)
        # Reconstruct the index in the DataFrame and get the latitude after deletion.
        NDBC_lat_aft = NDBC_lat_aft.reset_index(drop=True)
        Temp_lat=NDBC_lat_aft.values

        merged_array = np.concatenate([mean_result_axis_1, Temp_long.T,Temp_lat.T], axis=1)
        tsne = TSNE(n_components=2, random_state=30,perplexity=1) #Make sure random seed to be the same for the reproducibility.
        array_aft = tsne.fit_transform(merged_array)

        #GEt the new ADJ matrix.
        Adj_dist=adj_dist(pd.DataFrame(array_aft[:,0]),pd.DataFrame(array_aft[:,1]))
        capacities=np.ones((Adj_dist.shape[0], 1))
        capacities=capacities.flatten()
        split_line1 = int(result.shape[0] * 0.7)
        training_set = result[:split_line1, :]
        test_set = result[split_line1:, :]
        rand = np.random.RandomState(0) # Fixed random output, just an example when seed = 0.
        unknow_set = rand.choice(list(range(0,result.shape[1])),N_u,replace=True)#Stations to be predicted
        unknow_set = set(unknow_set)
        full_set = set(range(0,result.shape[1]))
        know_set = full_set - unknow_set
        training_set_s = training_set[:, list(know_set)]   # get the training data in the sample time period
        A_s = Adj_dist[:, list(know_set)][list(know_set), :]
        MAE_t, RMSE_t, R2_t, nrel_ignnk_res  = test_error_cap(STmodel, unknow_set, full_set,test_set, Adj_dist,h,capacities)
        print('Best model in the',j, '-th column is', MAE_t, RMSE_t, R2_t)

Best model in the 5 -th column is 0.40352601831183693 0.5088791407814893 -2.0018908772340307
Best model in the 6 -th column is 0.1411266781166549 0.1804560280265735 -2.0065000682516088
Best model in the 7 -th column is 0.1080074476969392 0.1418104682791045 -1.839235293152675
Best model in the 8 -th column is 0.07047534916285692 0.09903405982849459 -0.6631597810723096
Best model in the 9 -th column is 0.23394033180461932 0.2910954930464806 -2.6891267739028377
Best model in the 10 -th column is 0.12240251247475493 0.151003131106556 -2.662159706744909
Best model in the 11 -th column is 0.42374578401253 0.5127201027197579 -2.19312763841895
Best model in the 12 -th column is 0.03289810527367924 0.04290030633038398 -11.119319152080367
Best model in the 13 -th column is 0.5673032852716019 0.6638559782332828 -39.51562018379573
Best model in the 14 -th column is 0.44205008405445867 0.5270998671359972 -4.929568892482556
Best model in the 15 -th column is 0.5960648350147367 0.6273097089296188 -15

### Experiment 3.1: IGNNK-Training-dis-MI-based graph-3DGNN (Format:Metd-Adj-Only one defined graph-3DGNN)

In [7]:
#Subgraph generation module 1 --->Giving raw series, and generate a series of subgraph based on MI.
NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
NDBC_long=pd.DataFrame(Station_info.iloc[:,3])
NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
Adj_dist=adj_dist(NDBC_lat,NDBC_long)
j=5#J-th colum, start from 5 as the first feature.
X_raw_0=X_raw[:,j,:]#GET the first feature
print(X_raw_0.shape)
# Count the num of missing values in each column
missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
# print results
print("Incomplete data number in each column：", missing_counts_per_column)

#Get the index if the value is not zero
# Find the columns where missing values exist.
columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

# Get the column numbers when missing value exist.
missing_columns = np.where(columns_with_missing_data)[0]

# Print results
print("The column numbers with missing values are：", missing_columns)

# Get new data after deletion.
# Delete those columns (Stations) if there is not any features.
result = np.delete(X_raw_0, missing_columns, axis=1)
result = (result - np.min(result)) / (np.max(result) - np.min(result))
print(result.shape)#(8784, 93)

#Determine group num here
group_num=result.shape[1]-N_u

# Initialize MI_store
MI_store = np.zeros((result.shape[1], result.shape[1]))

# Calculate mutual information
for i in range(result.shape[1]):
    for j in range(result.shape[1]):
        MI_store[i, j] = calc_MI(result[:, i], result[:, j], 10)

# Return column numbers and corresponding mutual information
column_numbers = []
MI_values = []

for i in range(MI_store.shape[0]):
    row = MI_store[i, :]
    row_index = np.arange(MI_store.shape[0])
    row_index = np.delete(row_index, i)  # Exclude the current row's index
    sorted_indices = np.argsort(row[row_index])
    max_indices = row_index[sorted_indices[-group_num:][::-1]]  # Get largest column_num
    column_numbers.append(max_indices)
    MI_values.append([row[j] for j in max_indices])

# Output results
for i, (columns, values) in enumerate(zip(column_numbers, MI_values)):
    print(f"Row {i} - Column Numbers: {columns}, MI Values: {values}")

(8784, 103)
Incomplete data number in each column： [8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0 8784    0    0    0 8784    0    0    0
    0    0    0    0    0    0    0 8784 8784    0    0    0    0 8784
 8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0 8784    0    0    0    0    0    0    0 8784    0 8784
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
The column numbers with missing values are： [ 0 34 38 49 50 55 56 73 81 83]
(8784, 93)
Row 0 - Column Numbers: [ 1  2  4 79  5 20  3 77 21 17 83 85 78 87 91 12 15 31 11 43 19 44 35 30
 74 90 16 10 33 64 42 34 61 13 32 46 22 63 92 56 18  8 41], MI Values: [0.8272601132894586, 0.6901375697065202, 0.4673606483958732, 0.45757637448269683, 0.4363240338301644, 0.4190235175463517, 0.37428008133733837, 0.37263024544485734, 0.3321

In [8]:

from utils import load_data_MI

import torch
import numpy as np
import torch.optim as optim
from torch import nn
import matplotlib.pyplot as plt
from utils import *
import random
import pandas as pd
from basic_structure import IGNNK
from basic_process import *
import argparse
import sys
import os
import time
if __name__ == "__main__":
    """
    Model training
    """
    flags = parse_args(sys.argv[1:])
    dataset=flags.dataset
    n_o_n_m = flags.n_o
    h = flags.h
    z = flags.z
    K = flags.K
    n_m = flags.n_m
    n_u = flags.n_u
    max_iter = flags.max_iter
    learning_rate = flags.learning_rate
    E_maxvalue = flags.E_maxvalue
    batch_size = flags.batch_size
    to_plot = flags.to_plot

In [9]:
def test_error_new(STmodel, unknow_set, test_data, A_s, Missing0):
    """
    :param STmodel: The graph neural networks
    :unknow_set: The unknow locations for spatial prediction
    :test_data: The true value test_data of shape (test_num_timesteps, num_nodes)
    :A_s: The full adjacent matrix
    :Missing0: True: 0 in original datasets means missing data
    :return: NAE, MAPE and RMSE
    """
    unknow_set = set(unknow_set)
    time_dim = STmodel.time_dimension
    test_omask = np.ones(test_data.shape)
    if Missing0 == True:
        test_omask[test_data == 0] = 0
    test_inputs = (test_data * test_omask).astype('float32')
    test_inputs_s = test_inputs
    missing_index = np.ones(np.shape(test_data))
    missing_index[:, list(unknow_set)] = 0
    missing_index_s = missing_index
    o = np.zeros([test_data.shape[0]//time_dim*time_dim, test_inputs_s.shape[1]]) #Separate the test data into several h period
    for i in range(0, test_data.shape[0]//time_dim*time_dim, time_dim):
        inputs = test_inputs_s[i:i+time_dim, :]
        missing_inputs = missing_index_s[i:i+time_dim, :]
        T_inputs = inputs*missing_inputs
        flags = parse_args(sys.argv[1:])
        E_maxvalue = flags.E_maxvalue
        T_inputs = T_inputs/E_maxvalue
        T_inputs = np.expand_dims(T_inputs, axis = 0)
        T_inputs = torch.from_numpy(T_inputs.astype('float32'))
        A_q = torch.from_numpy((calculate_random_walk_matrix(A_s).T).astype('float32'))
        A_h = torch.from_numpy((calculate_random_walk_matrix(A_s.T).T).astype('float32'))
        imputation = STmodel(T_inputs, A_q, A_h)
        imputation = imputation.data.numpy()
        o[i:i+time_dim, :] = imputation[0, :, :]
    flags = parse_args(sys.argv[1:])
    dataset=flags.dataset
    if dataset == 'NREL':
        dataset=dataset
    else:
        o = o*E_maxvalue
    truth = test_inputs_s[0:test_set.shape[0]//time_dim*time_dim]
    o[missing_index_s[0:test_set.shape[0]//time_dim*time_dim] == 1] = truth[missing_index_s[0:test_set.shape[0]//time_dim*time_dim] == 1]
    test_mask =  1 - missing_index_s[0:test_set.shape[0]//time_dim*time_dim]
    if Missing0 == True:
        test_mask[truth == 0] = 0
        o[truth == 0] = 0
    MAE = np.sum(np.abs(o - truth))/np.sum( test_mask)
    RMSE = np.sqrt(np.sum((o - truth)*(o - truth))/np.sum( test_mask) )
    MAPE = np.sum(np.abs(o - truth)/(truth + 1e-5))/np.sum( test_mask)
    return MAE, RMSE, MAPE

In [10]:
# load dataset
A,X,training_set,test_set,unknow_set,full_set,know_set,training_set_s,A_s,capacity = load_data_MI('ndbc')

(1, 93)
A's shape is: (93, 93)
X's shape is: (93, 8784)
(8784, 103)
Incomplete data number in each column： [8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0 8784    0    0    0 8784    0    0    0
    0    0    0    0    0    0    0 8784 8784    0    0    0    0 8784
 8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0 8784    0    0    0    0    0    0    0 8784    0 8784
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
The column numbers with missing values are： [ 0 34 38 49 50 55 56 73 81 83]
(8784, 93)
Row 0 - Column Numbers: [ 1  2  4 79  5 20  3 77 21 17 83 85 78 87 91 12 15 31 11 43 19 44 35 30
 74 90 16 10 33 64 42 34 61 13 32 46 22 63 92 56 18  8 41], MI Values: [0.8272601132894586, 0.6901375697065202, 0.4673606483958732, 0.45757637448269683, 0.4363240338301644, 0.4190235175

In [11]:

def test_error(STmodel, unknow_set, test_data, A_s, Missing0):
    """
    :param STmodel: The graph neural networks
    :unknow_set: The unknow locations for spatial prediction
    :test_data: The true value test_data of shape (test_num_timesteps, num_nodes)
    :A_s: The full adjacent matrix
    :Missing0: True: 0 in original datasets means missing data
    :return: NAE, MAPE and RMSE
    """
    unknow_set = set(unknow_set)
    time_dim = STmodel.time_dimension
    test_omask = np.ones(test_data.shape)
    if Missing0 == True:
        test_omask[test_data == 0] = 0
    test_inputs = (test_data * test_omask).astype('float32')
    test_inputs_s = test_inputs
    missing_index = np.ones(np.shape(test_data))
    missing_index[:, list(unknow_set)] = 0
    missing_index_s = missing_index
    o = np.zeros([test_data.shape[0]//time_dim*time_dim, test_inputs_s.shape[1]]) #Separate the test data into several h period
    for i in range(0, test_data.shape[0]//time_dim*time_dim, time_dim):
        inputs = test_inputs_s[i:i+time_dim, :]
        missing_inputs = missing_index_s[i:i+time_dim, :]
        T_inputs = inputs*missing_inputs
        T_inputs = T_inputs/E_maxvalue
        T_inputs = np.expand_dims(T_inputs, axis = 0)
        T_inputs = torch.from_numpy(T_inputs.astype('float32'))
        A_q = torch.from_numpy((calculate_random_walk_matrix(A_s).T).astype('float32'))
        A_h = torch.from_numpy((calculate_random_walk_matrix(A_s.T).T).astype('float32'))
        imputation = STmodel(T_inputs, A_q, A_h)
        imputation = imputation.data.numpy()
        o[i:i+time_dim, :] = imputation[0, :, :]
    if dataset == 'NREL':
        o = o*capacities[None,:]
    else:
        o = o*E_maxvalue
    truth = test_inputs_s[0:test_set.shape[0]//time_dim*time_dim]
    o[missing_index_s[0:test_set.shape[0]//time_dim*time_dim] == 1] = truth[missing_index_s[0:test_set.shape[0]//time_dim*time_dim] == 1]
    test_mask =  1 - missing_index_s[0:test_set.shape[0]//time_dim*time_dim]
    if Missing0 == True:
        test_mask[truth == 0] = 0
        o[truth == 0] = 0
    MAE = np.sum(np.abs(o - truth))/np.sum( test_mask)
    RMSE = np.sqrt(np.sum((o - truth)*(o - truth))/np.sum( test_mask) )
    MAPE = np.sum(np.abs(o - truth)/(truth + 1e-5))/np.sum( test_mask)
    return MAE, RMSE, MAPE

In [12]:
#torch.save(STmodel.state_dict(), '/content/Spatial-interpolation/model/IGNNK_{:}_{:}iter_{:}.pth'.format(dataset,max_iter,time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))) # Save the model
torch.save(STmodel.state_dict(), '/home/xren451/rxb/phd/Spatial_interpolation/XBSPA/ModIGNNK/IGNNK_{:}iter_{:}.pth'.format(max_iter,time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))) # Save the model
STmodel.state_dict()

OrderedDict([('GNN1.Theta1',
              tensor([[-0.0565, -0.0106,  0.0122,  ..., -0.0457, -0.0026, -0.0830],
                      [ 0.0296, -0.0353, -0.0647,  ...,  0.0538,  0.0159, -0.0188],
                      [ 0.0064,  0.0306, -0.0617,  ...,  0.0932, -0.0761, -0.0050],
                      ...,
                      [ 0.0333, -0.0776, -0.0641,  ...,  0.0298,  0.0085,  0.0693],
                      [-0.0389,  0.0677, -0.0106,  ...,  0.0133, -0.0053, -0.0038],
                      [-0.0375,  0.0726, -0.0529,  ..., -0.0637, -0.0726, -0.0040]])),
             ('GNN1.bias',
              tensor([ 0.0516, -0.0006, -0.0130,  0.0840,  0.0660,  0.0443,  0.0540, -0.0252,
                      -0.0925, -0.0581, -0.0634,  0.0417,  0.0515,  0.0052,  0.0355, -0.0728,
                       0.0627, -0.0744, -0.0446, -0.0263, -0.0756, -0.0776, -0.0634, -0.0239,
                       0.0590,  0.0439, -0.0400,  0.0145,  0.0888,  0.0073,  0.0227, -0.0540,
                      -0.0872, -0.

Inference

In [13]:
 # Define model and train
STmodel = IGNNK(h, z, K)  # The graph neural networks
criterion = nn.MSELoss()
optimizer = optim.Adam(STmodel.parameters(), lr=learning_rate)
RMSE_list = []
MAE_list = []
MAPE_list = []
for epoch in range(max_iter):
    for i in range(training_set.shape[0]//(h * batch_size)):  #using time_length as reference to record test_error
        t_random = np.random.randint(0, high=(training_set_s.shape[0] - h), size=batch_size, dtype='l')
        know_mask = set(random.sample(range(0,training_set_s.shape[1]),n_o_n_m)) #sample n_o + n_m nodes
        feed_batch = []
        for j in range(batch_size):
            feed_batch.append(training_set_s[t_random[j]: t_random[j] + h, :][:, list(know_mask)]) #generate 8 time batches
        inputs = np.array(feed_batch)
        inputs_omask = np.ones(np.shape(inputs))
        if not dataset == 'NREL':
            inputs_omask[inputs == 0] = 0           # We found that there are irregular 0 values for METR-LA, so we treat those 0 values as missing data,
                                                    # For other datasets, it is not necessary to mask 0 values
        missing_index = np.ones((inputs.shape))
        for j in range(batch_size):
            missing_mask = random.sample(range(0,n_o_n_m),n_m) #Masked locations
            missing_index[j, :, missing_mask] = 0
        if dataset == 'NREL':
            Mf_inputs = inputs * inputs_omask * missing_index / capacities[:, None]
        else:
            Mf_inputs = inputs * inputs_omask * missing_index / E_maxvalue #normalize the value according to experience
        Mf_inputs = torch.from_numpy(Mf_inputs.astype('float32'))
        mask = torch.from_numpy(inputs_omask.astype('float32'))   #The reconstruction errors on irregular 0s are not used for training
        A_dynamic = A_s[list(know_mask), :][:, list(know_mask)]   #Obtain the dynamic adjacent matrix
        A_q = torch.from_numpy((calculate_random_walk_matrix(A_dynamic).T).astype('float32'))
        A_h = torch.from_numpy((calculate_random_walk_matrix(A_dynamic.T).T).astype('float32'))
        if dataset == 'NREL':
            outputs = torch.from_numpy(inputs/capacities[:, None])
        else:
            outputs = torch.from_numpy(inputs/E_maxvalue) #The label
        optimizer.zero_grad()
        X_res = STmodel(Mf_inputs, A_q, A_h)  #Obtain the reconstruction
        loss = criterion(X_res*mask, outputs*mask)
        loss.backward()
        optimizer.step()        #Errors backward
    if not dataset == 'NREL':
        flags = parse_args(sys.argv[1:])
        E_maxvalue = flags.E_maxvalue
        MAE_t, RMSE_t, MAPE_t = test_error(STmodel, unknow_set, test_set, A, E_maxvalue)
    else:
        MAE_t, RMSE_t, MAPE_t= test_error(STmodel, unknow_set, test_set, A,E_maxvalue)
    RMSE_list.append(RMSE_t)
    MAE_list.append(MAE_t)
    MAPE_list.append(MAPE_t)
    print(epoch, MAE_t, RMSE_t, MAPE_t)


0 1.6149224443523984 2.52955289554241 879.9701738180798
1 0.7877490398799057 1.3121267842842288 583.4288385626284
2 0.7352291173601594 1.2137979559888692 546.4334157530495
3 0.6888332073379644 1.125183008665543 510.9577476332256
4 0.641995979954657 1.0370048625319204 476.1958675750768
5 0.5977678576006255 0.9552627899781174 440.5831677556239
6 0.5567252959623211 0.8819417846689256 403.694635704134
7 0.5214402970455466 0.8134062462720038 378.4143287479428
8 0.47611740132883323 0.7391244577464788 341.8123945082279
9 0.4528726404718752 0.6848395230386973 325.185460233376
10 0.42336753970541635 0.6311392415894838 301.29145656436145
11 0.40836183967637163 0.5929155654196876 294.07332932421974
12 0.3877241664606062 0.5518707327273531 278.76200143827924
13 0.3662003603967178 0.5112564697171508 262.60388374292415
14 0.3524175333197208 0.48089751027599903 251.9919828560895
15 0.34367277984964323 0.457486771092555 250.77718985802
16 0.3337708614801255 0.4372562880959559 246.82251732890475
17 0.3

In [14]:
#Adj---Determined by tSNE
#Raw data and raw adjacency matrix without mapminmax
X_raw
# X_raw=normalize_3d_array(X_raw)

for j in range(5,X_raw.shape[1]-1):
    NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
    NDBC_long=pd.DataFrame(Station_info.iloc[:,3])
    NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
    Adj_dist=adj_dist(NDBC_lat,NDBC_long)
    X_raw_0=X_raw[:,j,:]#GET the first feature
    #print(X_raw_0.shape)
    # Count the num of missing values in each column
    missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
    # print results
    # print("Incomplete data number in each column：", missing_counts_per_column)

    #Get the index if the value is not zero
    # Find the columns where missing values exist.
    columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

    # Get the column numbers when missing value exist.
    missing_columns = np.where(columns_with_missing_data)[0]

    # Print results
    #print("The column numbers with missing values are：", missing_columns)

    #Get new data after deletion.
    #Delete those columns(Stations) if there is not any features.
    result = np.delete(X_raw_0, missing_columns, axis=1)
    result = (result - np.min(result)) / (np.max(result) - np.min(result))
    #Mean results---for SNE
    mean_result_axis_1=np.mean(result,axis=0)
    mean_result_axis_1=mean_result_axis_1.reshape(-1,1)

    #Get new adjacency matrix after deletion.
    NDBC_long=NDBC_long.transpose()
    NDBC_long_aft=NDBC_long.drop(columns=missing_columns)
    # Reconstruct the index in the DataFrame and get the longitude after deletion.
    NDBC_long_aft = NDBC_long_aft.reset_index(drop=True)
    print(NDBC_long_aft.shape)
    Temp_long=NDBC_long_aft.values

    NDBC_lat=NDBC_lat.transpose()
    NDBC_lat_aft=NDBC_lat.drop(columns=missing_columns)
    # Reconstruct the index in the DataFrame and get the latitude after deletion.
    NDBC_lat_aft = NDBC_lat_aft.reset_index(drop=True)
    print(NDBC_lat_aft.shape)
    Temp_lat=NDBC_lat_aft.values

    merged_array = np.concatenate([mean_result_axis_1, Temp_long.T,Temp_lat.T], axis=1)
    tsne = TSNE(n_components=2, random_state=30,perplexity=1) #Make sure random seed to be the same for the reproducibility.
    array_aft = tsne.fit_transform(merged_array)

    #GEt the new ADJ matrix.
    Adj_dist=adj_dist(pd.DataFrame(array_aft[:,0]),pd.DataFrame(array_aft[:,1]))
    capacities=np.ones((Adj_dist.shape[0], 1))
    capacities=capacities.flatten()
    split_line1 = int(result.shape[0] * 0.7)
    training_set = result[:split_line1, :]
    test_set = result[split_line1:, :]
    rand = np.random.RandomState(0) # Fixed random output, just an example when seed = 0.
    unknow_set = rand.choice(list(range(0,result.shape[1])),N_u,replace=True)#Stations to be predicted
    unknow_set = set(unknow_set)
    full_set = set(range(0,result.shape[1]))
    know_set = full_set - unknow_set
    training_set_s = training_set[:, list(know_set)]   # get the training data in the sample time period
    A_s = Adj_dist[:, list(know_set)][list(know_set), :]
    MAE_t, RMSE_t, R2_t, nrel_ignnk_res  = test_error_cap(STmodel, unknow_set, full_set,test_set, Adj_dist,h,capacities)
    print('Best model in the',j, '-th column is', MAE_t, RMSE_t, R2_t)

(1, 93)
(1, 93)
Best model in the 5 -th column is 0.3963973298993473 0.4965677424469117 -1.8583974139128197
(1, 96)
(1, 96)
Best model in the 6 -th column is 0.15896444868640408 0.19401845596661563 -2.4753977707385815
(1, 94)
(1, 94)
Best model in the 7 -th column is 0.1268784930729019 0.15618160090515343 -2.443852504411656
(1, 72)
(1, 72)
Best model in the 8 -th column is 0.08986223805704768 0.1183024403392515 -1.3732976811914015
(1, 72)
(1, 72)
Best model in the 9 -th column is 0.24969313192053522 0.29853945729939213 -2.880217721458145
(1, 65)
(1, 65)
Best model in the 10 -th column is 0.15529648691880432 0.17784814729769147 -4.080004130222068
(1, 40)
(1, 40)
Best model in the 11 -th column is 0.4535547051545998 0.5359839978249362 -2.4894681217241015
(1, 98)
(1, 98)
Best model in the 12 -th column is 0.052289027167505615 0.05821623130649479 -21.317503459953613
(1, 99)
(1, 99)
Best model in the 13 -th column is 0.538051544351029 0.6326040877792422 -35.790758184725895
(1, 77)
(1, 77)
B

Experiment 3.2: IGNNK-dis-MIsubgrh-3DGNN (Format:Metd-Adj-a node can constrct multiple subgraphs-3DGNN)

In [15]:
if __name__ == "__main__":
    """
    Model training
    """
    flags = parse_args(sys.argv[1:])
    dataset=flags.dataset
    n_o_n_m = flags.n_o
    h = flags.h
    z = flags.z
    K = flags.K
    n_m = flags.n_m
    n_u = flags.n_u
    max_iter = flags.max_iter
    learning_rate = flags.learning_rate
    E_maxvalue = flags.E_maxvalue
    batch_size = flags.batch_size
    to_plot = flags.to_plot

In [16]:
# load dataset
A,X,training_set,test_set,unknow_set,full_set,know_set,training_set_s,A_s,capacity = load_data_MI('ndbc')


(1, 93)
A's shape is: (93, 93)
X's shape is: (93, 8784)
(8784, 103)
Incomplete data number in each column： [8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0 8784    0    0    0 8784    0    0    0
    0    0    0    0    0    0    0 8784 8784    0    0    0    0 8784
 8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0 8784    0    0    0    0    0    0    0 8784    0 8784
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
The column numbers with missing values are： [ 0 34 38 49 50 55 56 73 81 83]
(8784, 93)
Row 0 - Column Numbers: [ 1  2  4 79  5 20  3 77 21 17 83 85 78 87 91 12 15 31 11 43 19 44 35 30
 74 90 16 10 33 64 42 34 61 13 32 46 22 63 92 56 18  8 41], MI Values: [0.8272601132894586, 0.6901375697065202, 0.4673606483958732, 0.45757637448269683, 0.4363240338301644, 0.4190235175

In [17]:
from __future__ import division
import torch
import numpy as np
import torch.optim as optim
from torch import nn
import matplotlib.pyplot as plt
from utils import *
import random
import pandas as pd
from basic_structure import IGNNK
from basic_process import *
import argparse
import sys
import os
import time
import pdb
 # Define model and train
STmodel = IGNNK(h, z, K)  # The graph neural networks
criterion = nn.MSELoss()
optimizer = optim.Adam(STmodel.parameters(), lr=learning_rate)
RMSE_list = []
MAE_list = []
MAPE_list = []
for epoch in range(max_iter):
    for i in range(training_set.shape[0]//(h * batch_size)):  #using time_length as reference to record test_error
        t_random = np.random.randint(0, high=(training_set_s.shape[0] - h), size=batch_size, dtype='l')
        know_mask = set(random.sample(range(0,training_set_s.shape[1]),n_o_n_m)) #sample n_o + n_m nodes
        feed_batch = []
        for j in range(batch_size):
            feed_batch.append(training_set_s[t_random[j]: t_random[j] + h, :][:, list(know_mask)]) #generate 8 time batches
        inputs = np.array(feed_batch)
        inputs_omask = np.ones(np.shape(inputs))
        if not dataset == 'NREL':
            inputs_omask[inputs == 0] = 0           # We found that there are irregular 0 values for METR-LA, so we treat those 0 values as missing data,
                                                    # For other datasets, it is not necessary to mask 0 values
        missing_index = np.ones((inputs.shape))
        for j in range(batch_size):
            missing_mask = random.sample(range(0,n_o_n_m),n_m) #Masked locations
            missing_index[j, :, missing_mask] = 0
        if dataset == 'NREL':
            Mf_inputs = inputs * inputs_omask * missing_index / capacities[:, None]
        else:
            Mf_inputs = inputs * inputs_omask * missing_index / E_maxvalue #normalize the value according to experience
        Mf_inputs = torch.from_numpy(Mf_inputs.astype('float32'))
        mask = torch.from_numpy(inputs_omask.astype('float32'))   #The reconstruction errors on irregular 0s are not used for training
        A_dynamic = A_s[list(know_mask), :][:, list(know_mask)]   #Obtain the dynamic adjacent matrix
        A_q = torch.from_numpy((calculate_random_walk_matrix(A_dynamic).T).astype('float32'))
        A_h = torch.from_numpy((calculate_random_walk_matrix(A_dynamic.T).T).astype('float32'))
        if dataset == 'NREL':
            outputs = torch.from_numpy(inputs/capacities[:, None])
        else:
            outputs = torch.from_numpy(inputs/E_maxvalue) #The label
        optimizer.zero_grad()
        X_res = STmodel(Mf_inputs, A_q, A_h)  #Obtain the reconstruction
        loss = criterion(X_res*mask, outputs*mask)
        loss.backward()
        optimizer.step()        #Errors backward
    if not dataset == 'NREL':
        MAE_t, RMSE_t, MAPE_t= test_error_new(STmodel, unknow_set, test_set, A, E_maxvalue)
    else:
        MAE_t, RMSE_t, MAPE_t= test_error_new(STmodel, unknow_set, test_set, A,E_maxvalue)
    RMSE_list.append(RMSE_t)
    MAE_list.append(MAE_t)
    MAPE_list.append(MAPE_t)
    print(epoch, MAE_t, RMSE_t, MAPE_t)

0 3.8821131800433846 5.738934539484558 1092.0942307341259
1 1.1407583723690777 1.6731743752803818 287.9954207174686
2 0.8410641901960044 1.2965538153562406 258.23561098245483
3 0.8042294406434464 1.2441544397055262 249.52167323996295
4 0.7690821432277976 1.191330875839847 242.3903908139965
5 0.7343648175601256 1.14138187062066 234.8007328148832
6 0.7081538409445882 1.0994694293175673 231.7585617444324
7 0.6744996838025722 1.0514412728108185 223.01737992427692
8 0.6417277524543326 1.0027990996717413 215.3846594854217
9 0.6226871449710973 0.9640124686764253 214.50746662669982
10 0.5951804885339449 0.9198348661887882 208.52073563708137
11 0.5751930741760104 0.8815028207899035 204.8874928431642
12 0.5487821315066919 0.838443613423531 197.00615331863636
13 0.5186939860098042 0.7942799926185324 185.32205643427218
14 0.5060034769538516 0.7643362966250455 186.4185160215653
15 0.49156862955579933 0.7323854964418166 185.2633752260314
16 0.4693259495285707 0.6959184948697634 177.99538015186326
17

In [18]:
#torch.save(STmodel.state_dict(), '/content/Spatial-interpolation/model/IGNNK_{:}_{:}iter_{:}.pth'.format(dataset,max_iter,time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))) # Save the model
torch.save(STmodel.state_dict(), '/home/xren451/rxb/phd/Spatial_interpolation/XBSPA/ModIGNNK/model/IGNNK_{:}iter_{:}.pth'.format(max_iter,time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))) # Save the model
STmodel.state_dict()

OrderedDict([('GNN1.Theta1',
              tensor([[-0.0440, -0.0673, -0.0438,  ..., -0.0012,  0.0591,  0.0705],
                      [ 0.0722, -0.0447,  0.0257,  ...,  0.0501,  0.0713,  0.0928],
                      [-0.0005, -0.0707,  0.0231,  ...,  0.0752,  0.0472, -0.0439],
                      ...,
                      [ 0.0833,  0.0616,  0.0626,  ..., -0.0187,  0.0021,  0.0268],
                      [ 0.0508,  0.0226,  0.0365,  ...,  0.0546, -0.0051, -0.0511],
                      [ 0.0192,  0.0621, -0.0710,  ..., -0.0031, -0.0175, -0.0462]])),
             ('GNN1.bias',
              tensor([-0.0814, -0.0938,  0.0013,  0.0030, -0.0714,  0.0373,  0.0529, -0.0372,
                      -0.0622,  0.0740,  0.0082,  0.0435,  0.0460, -0.0990,  0.0869, -0.0136,
                       0.0699,  0.0312, -0.0013,  0.0455, -0.0116, -0.0793,  0.0195, -0.0312,
                      -0.0824,  0.0638,  0.0369,  0.0384,  0.0679,  0.0545, -0.0176, -0.0291,
                       0.0313,  0.

In [19]:
#Subgraph generation module 1 --->Giving raw series, and generate a series of subgraph based on MI.
NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
NDBC_long=pd.DataFrame(Station_info.iloc[:,3])
NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
Adj_dist=adj_dist(NDBC_lat,NDBC_long)
j=5#J-th colum, start from 5 as the first feature.
X_raw_0=X_raw[:,j,:]#GET the first feature
print(X_raw_0.shape)
# Count the num of missing values in each column
missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
# print results
print("Incomplete data number in each column：", missing_counts_per_column)

#Get the index if the value is not zero
# Find the columns where missing values exist.
columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

# Get the column numbers when missing value exist.
missing_columns = np.where(columns_with_missing_data)[0]

# Print results
print("The column numbers with missing values are：", missing_columns)

# Get new data after deletion.
# Delete those columns (Stations) if there is not any features.
result = np.delete(X_raw_0, missing_columns, axis=1)
result = (result - np.min(result)) / (np.max(result) - np.min(result))
print(result.shape)#(8784, 93)

#Determine group num here
group_num=result.shape[1]-N_u

# Initialize MI_store
MI_store = np.zeros((result.shape[1], result.shape[1]))

# Calculate mutual information
for i in range(result.shape[1]):
    for j in range(result.shape[1]):
        MI_store[i, j] = calc_MI(result[:, i], result[:, j], 10)

# Return column numbers and corresponding mutual information
column_numbers = []
MI_values = []

for i in range(MI_store.shape[0]):
    row = MI_store[i, :]
    row_index = np.arange(MI_store.shape[0])
    row_index = np.delete(row_index, i)  # Exclude the current row's index
    sorted_indices = np.argsort(row[row_index])
    max_indices = row_index[sorted_indices[-group_num:][::-1]]  # Get largest column_num
    column_numbers.append(max_indices)
    MI_values.append([row[j] for j in max_indices])

# Output results
for i, (columns, values) in enumerate(zip(column_numbers, MI_values)):
    print(f"Row {i} - Column Numbers: {columns}, MI Values: {values}")

(8784, 103)
Incomplete data number in each column： [8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0 8784    0    0    0 8784    0    0    0
    0    0    0    0    0    0    0 8784 8784    0    0    0    0 8784
 8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0 8784    0    0    0    0    0    0    0 8784    0 8784
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
The column numbers with missing values are： [ 0 34 38 49 50 55 56 73 81 83]
(8784, 93)
Row 0 - Column Numbers: [ 1  2  4 79  5 20  3 77 21 17 83 85 78 87 91 12 15 31 11 43 19 44 35 30
 74 90 16 10 33 64 42 34 61 13 32 46 22 63 92 56 18  8 41], MI Values: [0.8272601132894586, 0.6901375697065202, 0.4673606483958732, 0.45757637448269683, 0.4363240338301644, 0.4190235175463517, 0.37428008133733837, 0.37263024544485734, 0.3321

In [20]:
A,X,training_set,test_set,unknow_set,full_set,know_set,training_set_s,A_s,capacity,unknow_set_2D,know_set_2D,training_set_s_3D = load_data_MI_3D('ndbc')

(1, 93)
A's shape is: (93, 93)
X's shape is: (93, 8784)
(8784, 103)
Incomplete data number in each column： [8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0 8784    0    0    0 8784    0    0    0
    0    0    0    0    0    0    0 8784 8784    0    0    0    0 8784
 8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0 8784    0    0    0    0    0    0    0 8784    0 8784
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
The column numbers with missing values are： [ 0 34 38 49 50 55 56 73 81 83]
(8784, 93)
Row 0 - Column Numbers: [ 1  2  4 79  5 20  3 77 21 17 83 85 78 87 91 12 15 31 11 43 19 44 35 30
 74 90 16 10 33 64 42 34 61 13 32 46 22 63 92 56 18  8 41], MI Values: [0.8272601132894586, 0.6901375697065202, 0.4673606483958732, 0.45757637448269683, 0.4363240338301644, 0.4190235175

In [21]:
training_set_s_3D.shape

(93, 6148, 43)

In [ ]:
unknow_set_2D
know_set_2D

In [26]:
if __name__ == "__main__":
    """
    Model training
    """
    flags = parse_args(sys.argv[1:])
    dataset=flags.dataset
    n_o_n_m = flags.n_o
    h = flags.h
    z = flags.z
    K = flags.K
    n_m = flags.n_m
    n_u = flags.n_u
    max_iter = flags.max_iter
    learning_rate = flags.learning_rate
    E_maxvalue = flags.E_maxvalue
    batch_size = flags.batch_size
    to_plot = flags.to_plot

# Define model and train
STmodel = IGNNK(h, z, K)  # The graph neural networks
criterion = nn.MSELoss()
optimizer = optim.Adam(STmodel.parameters(), lr=learning_rate)
RMSE_list = []
MAE_list = []
MAPE_list = []
for epoch in range(max_iter):
    for i in range(training_set.shape[0]//(h * batch_size)):  #using time_length as reference to record test_error
        t_random = np.random.randint(0, high=(training_set_s.shape[0] - h), size=batch_size, dtype='l')
        know_mask = set(random.sample(range(0,training_set_s.shape[1]),n_o_n_m)) #sample n_o + n_m nodes
        feed_batch = []
        for j in range(batch_size):
            feed_batch.append(training_set_s[t_random[j]: t_random[j] + h, :][:, list(know_mask)]) #generate 8 time batches
        inputs = np.array(feed_batch)
        inputs_omask = np.ones(np.shape(inputs))
        if not dataset == 'NREL':
            inputs_omask[inputs == 0] = 0           # We found that there are irregular 0 values for METR-LA, so we treat those 0 values as missing data,
                                                    # For other datasets, it is not necessary to mask 0 values
        missing_index = np.ones((inputs.shape))
        for j in range(batch_size):
            missing_mask = random.sample(range(0,n_o_n_m),n_m) #Masked locations
            missing_index[j, :, missing_mask] = 0
        if dataset == 'NREL':
            Mf_inputs = inputs * inputs_omask * missing_index / capacities[:, None]
        else:
            Mf_inputs = inputs * inputs_omask * missing_index / E_maxvalue #normalize the value according to experience
        Mf_inputs = torch.from_numpy(Mf_inputs.astype('float32'))
        mask = torch.from_numpy(inputs_omask.astype('float32'))   #The reconstruction errors on irregular 0s are not used for training
        A_dynamic = A_s[list(know_mask), :][:, list(know_mask)]   #Obtain the dynamic adjacent matrix
        A_q = torch.from_numpy((calculate_random_walk_matrix(A_dynamic).T).astype('float32'))
        A_h = torch.from_numpy((calculate_random_walk_matrix(A_dynamic.T).T).astype('float32'))
        if dataset == 'NREL':
            outputs = torch.from_numpy(inputs/capacities[:, None])
        else:
            outputs = torch.from_numpy(inputs/E_maxvalue) #The label
        optimizer.zero_grad()
        X_res = STmodel(Mf_inputs, A_q, A_h)  #Obtain the reconstruction
        loss = criterion(X_res*mask, outputs*mask)
        loss.backward()
        optimizer.step()        #Errors backward
    if not dataset == 'NREL':
        flags = parse_args(sys.argv[1:])
        E_maxvalue = flags.E_maxvalue
        MAE_t, RMSE_t, MAPE_t = test_error_new(STmodel, unknow_set, test_set, A, E_maxvalue)
    else:
        MAE_t, RMSE_t, MAPE_t= test_error_new(STmodel, unknow_set, test_set, A,E_maxvalue)
    RMSE_list.append(RMSE_t)
    MAE_list.append(MAE_t)
    MAPE_list.append(MAPE_t)
    print(epoch, MAE_t, RMSE_t, MAPE_t)


#Adj---Determined by tSNE
#Raw data and raw adjacency matrix without mapminmax
X_raw
# X_raw=normalize_3d_array(X_raw)

for j in range(5,X_raw.shape[1]-1):
    NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
    NDBC_long=pd.DataFrame(Station_info.iloc[:,3])
    NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
    Adj_dist=adj_dist(NDBC_lat,NDBC_long)
    X_raw_0=X_raw[:,j,:]#GET the first feature
    #print(X_raw_0.shape)
    # Count the num of missing values in each column
    missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
    # print results
    # print("Incomplete data number in each column：", missing_counts_per_column)

    #Get the index if the value is not zero
    # Find the columns where missing values exist.
    columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

    # Get the column numbers when missing value exist.
    missing_columns = np.where(columns_with_missing_data)[0]

    # Print results
    #print("The column numbers with missing values are：", missing_columns)

    #Get new data after deletion.
    #Delete those columns(Stations) if there is not any features.
    result = np.delete(X_raw_0, missing_columns, axis=1)
    result = (result - np.min(result)) / (np.max(result) - np.min(result))
    #Mean results---for SNE
    mean_result_axis_1=np.mean(result,axis=0)
    mean_result_axis_1=mean_result_axis_1.reshape(-1,1)

    #Get new adjacency matrix after deletion.
    NDBC_long=NDBC_long.transpose()
    NDBC_long_aft=NDBC_long.drop(columns=missing_columns)
    # Reconstruct the index in the DataFrame and get the longitude after deletion.
    NDBC_long_aft = NDBC_long_aft.reset_index(drop=True)
    print(NDBC_long_aft.shape)
    Temp_long=NDBC_long_aft.values

    NDBC_lat=NDBC_lat.transpose()
    NDBC_lat_aft=NDBC_lat.drop(columns=missing_columns)
    # Reconstruct the index in the DataFrame and get the latitude after deletion.
    NDBC_lat_aft = NDBC_lat_aft.reset_index(drop=True)
    print(NDBC_lat_aft.shape)
    Temp_lat=NDBC_lat_aft.values

    merged_array = np.concatenate([mean_result_axis_1, Temp_long.T,Temp_lat.T], axis=1)
    tsne = TSNE(n_components=2, random_state=30,perplexity=1) #Make sure random seed to be the same for the reproducibility.
    array_aft = tsne.fit_transform(merged_array)

    #GEt the new ADJ matrix.
    Adj_dist=adj_dist(pd.DataFrame(array_aft[:,0]),pd.DataFrame(array_aft[:,1]))
    capacities=np.ones((Adj_dist.shape[0], 1))
    capacities=capacities.flatten()
    split_line1 = int(result.shape[0] * 0.7)
    training_set = result[:split_line1, :]
    test_set = result[split_line1:, :]
    rand = np.random.RandomState(0) # Fixed random output, just an example when seed = 0.
    unknow_set = rand.choice(list(range(0,result.shape[1])),N_u,replace=True)#Stations to be predicted
    unknow_set = set(unknow_set)
    full_set = set(range(0,result.shape[1]))
    know_set = full_set - unknow_set
    training_set_s = training_set[:, list(know_set)]   # get the training data in the sample time period
    A_s = Adj_dist[:, list(know_set)][list(know_set), :]
    MAE_t, RMSE_t, R2_t, nrel_ignnk_res  = test_error_cap(STmodel, unknow_set, full_set,test_set, Adj_dist,h,capacities)
    print('Best model in the',j, '-th column is', MAE_t, RMSE_t, R2_t)

0 2.651729708605904 3.7956419609937453 536.2934281503201
1 1.016575022307287 1.698995022539805 207.46193650197083
2 0.970609496399443 1.6264375340799797 202.7414339968379
3 0.9370821843085656 1.5669263176289367 196.96721365542646
4 0.8920758921963099 1.4964119351814464 187.41058187503208
5 0.8559166753035169 1.4303165874599781 181.52387070809758
6 0.8194022884053219 1.3645084017992435 177.4558889448902
7 0.7802135057901641 1.300850298598197 168.3189778508396
8 0.746877315187496 1.2403882968595619 162.58862854489266
9 0.7180243509588887 1.184085372380192 159.37613515440384
10 0.6838944854046982 1.127367374102417 152.03956999932592
11 0.6508450035564441 1.0677169742097425 148.76547198815862
12 0.6170224747284075 1.0086684916956152 141.15633374776584
13 0.5916006662357839 0.9627470379198134 133.40878807770804
14 0.5688518122557337 0.9106339802983228 137.67039939447687
15 0.5360082746062884 0.8556460198174178 126.08209702115839
16 0.5101850801051486 0.805955097430131 123.41884293998613
17 

In [ ]:
if __name__ == "__main__":
    """
    Model training
    """
    flags = parse_args(sys.argv[1:])
    dataset=flags.dataset
    n_o_n_m = flags.n_o
    h = flags.h
    z = flags.z
    K = flags.K
    n_m = flags.n_m
    n_u = flags.n_u
    max_iter = flags.max_iter
    learning_rate = flags.learning_rate
    E_maxvalue = flags.E_maxvalue
    batch_size = flags.batch_size
    to_plot = flags.to_plot

# Define model and train
STmodel = IGNNK(h, z, K)  # The graph neural networks
criterion = nn.MSELoss()
optimizer = optim.Adam(STmodel.parameters(), lr=learning_rate)
RMSE_list = []
MAE_list = []
MAPE_list = []
for epoch in range(max_iter):
    for i in range(training_set.shape[0]//(h * batch_size)):  #using time_length as reference to record test_error
        t_random = np.random.randint(0, high=(training_set_s.shape[0] - h), size=batch_size, dtype='l')
        know_mask = set(random.sample(range(0,training_set_s.shape[1]),n_o_n_m)) #sample n_o + n_m nodes
        feed_batch = []
        for j in range(batch_size):
            feed_batch.append(training_set_s[t_random[j]: t_random[j] + h, :][:, list(know_mask)]) #generate 8 time batches
        inputs = np.array(feed_batch)
        inputs_omask = np.ones(np.shape(inputs))
        if not dataset == 'NREL':
            inputs_omask[inputs == 0] = 0           # We found that there are irregular 0 values for METR-LA, so we treat those 0 values as missing data,
                                                    # For other datasets, it is not necessary to mask 0 values
        missing_index = np.ones((inputs.shape))
        for j in range(batch_size):
            missing_mask = random.sample(range(0,n_o_n_m),n_m) #Masked locations
            missing_index[j, :, missing_mask] = 0
        if dataset == 'NREL':
            Mf_inputs = inputs * inputs_omask * missing_index / capacities[:, None]
        else:
            Mf_inputs = inputs * inputs_omask * missing_index / E_maxvalue #normalize the value according to experience
        Mf_inputs = torch.from_numpy(Mf_inputs.astype('float32'))
        mask = torch.from_numpy(inputs_omask.astype('float32'))   #The reconstruction errors on irregular 0s are not used for training
        A_dynamic = A_s[list(know_mask), :][:, list(know_mask)]   #Obtain the dynamic adjacent matrix
        A_q = torch.from_numpy((calculate_random_walk_matrix(A_dynamic).T).astype('float32'))
        A_h = torch.from_numpy((calculate_random_walk_matrix(A_dynamic.T).T).astype('float32'))
        if dataset == 'NREL':
            outputs = torch.from_numpy(inputs/capacities[:, None])
        else:
            outputs = torch.from_numpy(inputs/E_maxvalue) #The label
        optimizer.zero_grad()
        X_res = STmodel(Mf_inputs, A_q, A_h)  #Obtain the reconstruction
        loss = criterion(X_res*mask, outputs*mask)
        loss.backward()
        optimizer.step()        #Errors backward
    if not dataset == 'NREL':
        flags = parse_args(sys.argv[1:])
        E_maxvalue = flags.E_maxvalue
        MAE_t, RMSE_t, MAPE_t = test_error_new(STmodel, unknow_set, test_set, A, E_maxvalue)
    else:
        MAE_t, RMSE_t, MAPE_t= test_error_new(STmodel, unknow_set, test_set, A,E_maxvalue)
    RMSE_list.append(RMSE_t)
    MAE_list.append(MAE_t)
    MAPE_list.append(MAPE_t)
    print(epoch, MAE_t, RMSE_t, MAPE_t)


#Adj---Determined by tSNE
#Raw data and raw adjacency matrix without mapminmax
X_raw
# X_raw=normalize_3d_array(X_raw)

for j in range(5,X_raw.shape[1]-1):
    NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
    NDBC_long=pd.DataFrame(Station_info.iloc[:,3])
    NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
    Adj_dist=adj_dist(NDBC_lat,NDBC_long)
    X_raw_0=X_raw[:,j,:]#GET the first feature
    #print(X_raw_0.shape)
    # Count the num of missing values in each column
    missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
    # print results
    # print("Incomplete data number in each column：", missing_counts_per_column)

    #Get the index if the value is not zero
    # Find the columns where missing values exist.
    columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

    # Get the column numbers when missing value exist.
    missing_columns = np.where(columns_with_missing_data)[0]

    # Print results
    #print("The column numbers with missing values are：", missing_columns)

    #Get new data after deletion.
    #Delete those columns(Stations) if there is not any features.
    result = np.delete(X_raw_0, missing_columns, axis=1)
    result = (result - np.min(result)) / (np.max(result) - np.min(result))
    #Mean results---for SNE
    mean_result_axis_1=np.mean(result,axis=0)
    mean_result_axis_1=mean_result_axis_1.reshape(-1,1)

    #Get new adjacency matrix after deletion.
    NDBC_long=NDBC_long.transpose()
    NDBC_long_aft=NDBC_long.drop(columns=missing_columns)
    # Reconstruct the index in the DataFrame and get the longitude after deletion.
    NDBC_long_aft = NDBC_long_aft.reset_index(drop=True)
    print(NDBC_long_aft.shape)
    Temp_long=NDBC_long_aft.values

    NDBC_lat=NDBC_lat.transpose()
    NDBC_lat_aft=NDBC_lat.drop(columns=missing_columns)
    # Reconstruct the index in the DataFrame and get the latitude after deletion.
    NDBC_lat_aft = NDBC_lat_aft.reset_index(drop=True)
    print(NDBC_lat_aft.shape)
    Temp_lat=NDBC_lat_aft.values

    merged_array = np.concatenate([mean_result_axis_1, Temp_long.T,Temp_lat.T], axis=1)
    tsne = TSNE(n_components=2, random_state=30,perplexity=1) #Make sure random seed to be the same for the reproducibility.
    array_aft = tsne.fit_transform(merged_array)

    #GEt the new ADJ matrix.
    Adj_dist=adj_dist(pd.DataFrame(array_aft[:,0]),pd.DataFrame(array_aft[:,1]))
    capacities=np.ones((Adj_dist.shape[0], 1))
    capacities=capacities.flatten()
    split_line1 = int(result.shape[0] * 0.7)
    training_set = result[:split_line1, :]
    test_set = result[split_line1:, :]
    rand = np.random.RandomState(0) # Fixed random output, just an example when seed = 0.
    unknow_set = rand.choice(list(range(0,result.shape[1])),N_u,replace=True)#Stations to be predicted
    unknow_set = set(unknow_set)
    full_set = set(range(0,result.shape[1]))
    know_set = full_set - unknow_set
    training_set_s = training_set[:, list(know_set)]   # get the training data in the sample time period
    A_s = Adj_dist[:, list(know_set)][list(know_set), :]
    MAE_t, RMSE_t, R2_t, nrel_ignnk_res  = test_error_cap(STmodel, unknow_set, full_set,test_set, Adj_dist,h,capacities)
    print('Best model in the',j, '-th column is', MAE_t, RMSE_t, R2_t)

In [27]:


import torch
import torch.nn as nn
from torch.distributions.normal import Normal
import numpy as np


class SparseDispatcher(object):
    """Helper for implementing a mixture of experts.
    The purpose of this class is to create input minibatches for the
    experts and to combine the results of the experts to form a unified
    output tensor.
    There are two functions:
    dispatch - take an input Tensor and create input Tensors for each expert.
    combine - take output Tensors from each expert and form a combined output
      Tensor.  Outputs from different experts for the same batch element are
      summed together, weighted by the provided "gates".
    The class is initialized with a "gates" Tensor, which specifies which
    batch elements go to which experts, and the weights to use when combining
    the outputs.  Batch element b is sent to expert e iff gates[b, e] != 0.
    The inputs and outputs are all two-dimensional [batch, depth].
    Caller is responsible for collapsing additional dimensions prior to
    calling this class and reshaping the output to the original shape.
    See common_layers.reshape_like().
    Example use:
    gates: a float32 `Tensor` with shape `[batch_size, num_experts]`
    inputs: a float32 `Tensor` with shape `[batch_size, input_size]`
    experts: a list of length `num_experts` containing sub-networks.
    dispatcher = SparseDispatcher(num_experts, gates)
    expert_inputs = dispatcher.dispatch(inputs)
    expert_outputs = [experts[i](expert_inputs[i]) for i in range(num_experts)]
    outputs = dispatcher.combine(expert_outputs)
    The preceding code sets the output for a particular example b to:
    output[b] = Sum_i(gates[b, i] * experts[i](inputs[b]))
    This class takes advantage of sparsity in the gate matrix by including in the
    `Tensor`s for expert i only the batch elements for which `gates[b, i] > 0`.
    """

    def __init__(self, num_experts, gates):
        """Create a SparseDispatcher."""

        self._gates = gates
        self._num_experts = num_experts
        # sort experts
        sorted_experts, index_sorted_experts = torch.nonzero(gates).sort(0)
        # drop indices
        _, self._expert_index = sorted_experts.split(1, dim=1)
        # get according batch index for each expert
        self._batch_index = torch.nonzero(gates)[index_sorted_experts[:, 1], 0]
        # calculate num samples that each expert gets
        self._part_sizes = (gates > 0).sum(0).tolist()
        # expand gates to match with self._batch_index
        gates_exp = gates[self._batch_index.flatten()]
        self._nonzero_gates = torch.gather(gates_exp, 1, self._expert_index)

    def dispatch(self, inp, edge_index, edge_attr):
        """Create one input Tensor for each expert.
        The `Tensor` for a expert `i` contains the slices of `inp` corresponding
        to the batch elements `b` where `gates[b, i] > 0`.
        Args:
          inp: a `Tensor` of shape "[batch_size, <extra_input_dims>]`
        Returns:
          a list of `num_experts` `Tensor`s with shapes
            `[expert_batch_size_i, <extra_input_dims>]`.
        """

        # Note by Haotao:
        # self._batch_index: shape=(N_batch). The re-order indices from 0 to N_batch-1.
        # inp_exp: shape=inp.shape. The input Tensor re-ordered by self._batch_index along the batch dimension.
        # self._part_sizes: shape=(N_experts), sum=N_batch. self._part_sizes[i] is the number of samples routed towards expert[i].
        # return value: list [Tensor with shape[0]=self._part_sizes[i] for i in range(N_experts)] 

        # assigns samples to experts whose gate is nonzero

        # expand according to batch index so we can just split by _part_sizes
        inp_exp = inp[self._batch_index].squeeze(1)
        edge_index_exp = edge_index[:,self._batch_index]
        edge_attr_exp = edge_attr[self._batch_index]
        return torch.split(inp_exp, self._part_sizes, dim=0), torch.split(edge_index_exp, self._part_sizes, dim=1), torch.split(edge_attr_exp, self._part_sizes, dim=0)

    def combine(self, expert_out, multiply_by_gates=True):
        """Sum together the expert output, weighted by the gates.
        The slice corresponding to a particular batch element `b` is computed
        as the sum over all experts `i` of the expert output, weighted by the
        corresponding gate values.  If `multiply_by_gates` is set to False, the
        gate values are ignored.
        Args:
          expert_out: a list of `num_experts` `Tensor`s, each with shape
            `[expert_batch_size_i, <extra_output_dims>]`.
          multiply_by_gates: a boolean
        Returns:
          a `Tensor` with shape `[batch_size, <extra_output_dims>]`.
        """
        # apply exp to expert outputs, so we are not longer in log space
        stitched = torch.cat(expert_out, 0).exp()

        if multiply_by_gates:
            stitched = stitched.mul(self._nonzero_gates)
        zeros = torch.zeros(self._gates.size(0), expert_out[-1].size(1), requires_grad=True, device=stitched.device)
        # combine samples that have been processed by the same k experts
        combined = zeros.index_add(0, self._batch_index, stitched.float())
        # add eps to all zero values in order to avoid nans when going back to log space
        combined[combined == 0] = np.finfo(float).eps
        # back to log space
        return combined.log()

    def expert_to_gates(self):
        """Gate values corresponding to the examples in the per-expert `Tensor`s.
        Returns:
          a list of `num_experts` one-dimensional `Tensor`s with type `tf.float32`
              and shapes `[expert_batch_size_i]`
        """
        # split nonzero gates for each expert
        return torch.split(self._nonzero_gates, self._part_sizes, dim=0)

class MoE(nn.Module):

    """Call a Sparsely gated mixture of experts layer with 1-layer Feed-Forward networks as experts.
    Args:
    input_size: integer - size of the input
    output_size: integer - size of the input
    num_experts: an integer - number of experts
    hidden_size: an integer - hidden size of the experts
    noisy_gating: a boolean
    k: an integer - how many experts to use for each batch element
    """

    def __init__(self, input_size, output_size, num_experts, experts_conv, experts_bn, noisy_gating=True, k=4, coef=1e-2, num_experts_1hop=None):
        super(MoE, self).__init__()
        self.noisy_gating = noisy_gating
        self.num_experts = num_experts
        self.output_size = output_size
        self.input_size = input_size
        self.k = k
        self.loss_coef = coef
        if not num_experts_1hop:
            self.num_experts_1hop = num_experts # by default, all experts are hop-1 experts.
        else:
            assert num_experts_1hop <= num_experts
            self.num_experts_1hop = num_experts_1hop
        # instantiate experts
        # self.experts = nn.ModuleList([MLP(self.input_size, self.output_size, self.hidden_size) for i in range(self.num_experts)])
        self.experts_conv = experts_conv
        self.experts_bn = experts_bn
        self.w_gate = nn.Parameter(torch.zeros(input_size, num_experts), requires_grad=True)
        self.w_noise = nn.Parameter(torch.zeros(input_size, num_experts), requires_grad=True)

        self.softplus = nn.Softplus()
        self.softmax = nn.Softmax(1)
        self.register_buffer("mean", torch.tensor([0.0]))
        self.register_buffer("std", torch.tensor([1.0]))
        assert(self.k <= self.num_experts)

    def cv_squared(self, x):
        """The squared coefficient of variation of a sample.
        Useful as a loss to encourage a positive distribution to be more uniform.
        Epsilons added for numerical stability.
        Returns 0 for an empty Tensor.
        Args:
        x: a `Tensor`.
        Returns:
        a `Scalar`.
        """
        eps = 1e-10
        # if only num_experts = 1

        if x.shape[0] == 1:
            return torch.tensor([0], device=x.device, dtype=x.dtype)
        return x.float().var() / (x.float().mean()**2 + eps)

    def _gates_to_load(self, gates):
        """Compute the true load per expert, given the gates.
        The load is the number of examples for which the corresponding gate is >0.
        Args:
        gates: a `Tensor` of shape [batch_size, n]
        Returns:
        a float32 `Tensor` of shape [n]
        """
        return (gates > 0).sum(0)

    def _prob_in_top_k(self, clean_values, noisy_values, noise_stddev, noisy_top_values):
        """Helper function to NoisyTopKGating.
        Computes the probability that value is in top k, given different random noise.
        This gives us a way of backpropagating from a loss that balances the number
        of times each expert is in the top k experts per example.
        In the case of no noise, pass in None for noise_stddev, and the result will
        not be differentiable.
        Args:
        clean_values: a `Tensor` of shape [batch, n].
        noisy_values: a `Tensor` of shape [batch, n].  Equal to clean values plus
          normally distributed noise with standard deviation noise_stddev.
        noise_stddev: a `Tensor` of shape [batch, n], or None
        noisy_top_values: a `Tensor` of shape [batch, m].
           "values" Output of tf.top_k(noisy_top_values, m).  m >= k+1
        Returns:
        a `Tensor` of shape [batch, n].
        """
        batch = clean_values.size(0)
        m = noisy_top_values.size(1)
        top_values_flat = noisy_top_values.flatten()

        threshold_positions_if_in = torch.arange(batch, device=clean_values.device) * m + self.k
        threshold_if_in = torch.unsqueeze(torch.gather(top_values_flat, 0, threshold_positions_if_in), 1)
        is_in = torch.gt(noisy_values, threshold_if_in)
        threshold_positions_if_out = threshold_positions_if_in - 1
        threshold_if_out = torch.unsqueeze(torch.gather(top_values_flat, 0, threshold_positions_if_out), 1)
        # is each value currently in the top k.
        normal = Normal(self.mean, self.std)
        prob_if_in = normal.cdf((clean_values - threshold_if_in)/noise_stddev)
        prob_if_out = normal.cdf((clean_values - threshold_if_out)/noise_stddev)
        prob = torch.where(is_in, prob_if_in, prob_if_out)
        return prob

    def noisy_top_k_gating(self, x, train, noise_epsilon=1e-2):
        """Noisy top-k gating.
          See paper: https://arxiv.org/abs/1701.06538.
          Args:
            x: input Tensor with shape [batch_size, input_size]
            train: a boolean - we only add noise at training time.
            noise_epsilon: a float
          Returns:
            gates: a Tensor with shape [batch_size, num_experts]
            load: a Tensor with shape [num_experts]
        """
        clean_logits = x @ self.w_gate
        if self.noisy_gating and train:
            raw_noise_stddev = x @ self.w_noise
            noise_stddev = ((self.softplus(raw_noise_stddev) + noise_epsilon))
            noisy_logits = clean_logits + (torch.randn_like(clean_logits) * noise_stddev)
            logits = noisy_logits
        else:
            logits = clean_logits

        # calculate topk + 1 that will be needed for the noisy gates
        top_logits, top_indices = logits.topk(min(self.k + 1, self.num_experts), dim=1)
        top_k_logits = top_logits[:, :self.k]
        top_k_indices = top_indices[:, :self.k]
        top_k_gates = self.softmax(top_k_logits)

        zeros = torch.zeros_like(logits, requires_grad=True)
        gates = zeros.scatter(1, top_k_indices, top_k_gates)

        if self.noisy_gating and self.k < self.num_experts and train:
            load = (self._prob_in_top_k(clean_logits, noisy_logits, noise_stddev, top_logits)).sum(0)
        else:
            load = self._gates_to_load(gates)
        return gates, load

    def forward(self, x, edge_index, edge_attr, edge_index_2hop=None, edge_attr_2hop=None):
        """Args:
        x: tensor shape [batch_size, input_size]
        train: a boolean scalar.
        loss_coef: a scalar - multiplier on load-balancing losses

        Returns:
        y: a tensor with shape [batch_size, output_size].
        extra_training_loss: a scalar.  This should be added into the overall
        training loss of the model.  The backpropagation of this loss
        encourages all experts to be approximately equally used across a batch.
        """
        gates, load = self.noisy_top_k_gating(x, self.training)
        # calculate importance loss
        importance = gates.sum(0)
        #
        loss = self.cv_squared(importance) + self.cv_squared(load)
        loss *= self.loss_coef



        expert_outputs = []
        for i in range(self.num_experts):
            if i < self.num_experts_1hop:
                expert_i_output = self.experts_conv[i](x, edge_index, edge_attr)
            else:
                expert_i_output = self.experts_conv[i](x, edge_index_2hop, edge_attr_2hop)
            expert_i_output = self.experts_bn[i](expert_i_output)
            expert_outputs.append(expert_i_output)
        expert_outputs = torch.stack(expert_outputs, dim=1) # shape=[num_nodes, num_experts, d_feature]

        # gates: shape=[num_nodes, num_experts]
        y = gates.unsqueeze(dim=-1) * expert_outputs
        y = y.mean(dim=1)

        return y, loss

if __name__ == '__main__':
    class MLP(nn.Module):
        def __init__(self, input_size, output_size, hidden_size):
            super(MLP, self).__init__()
            self.fc1 = nn.Linear(input_size, hidden_size)
            self.fc2 = nn.Linear(hidden_size, output_size)
            self.relu = nn.ReLU()
            self.soft = nn.Softmax(1)

        def forward(self, x):
            out = self.fc1(x)
            out = self.relu(out)
            out = self.fc2(out)
            out = self.soft(out)
            return out

    moe_model = MoE(10, 10, 3, [MLP(10,10,10), MLP(10,10,10), MLP(10,10,10)], k=1)
    x = torch.ones((8,10))
    h = moe_model(x)
    print(h.shape)

TypeError: __init__() missing 1 required positional argument: 'experts_bn'